# Pre proccessing

In [2]:
import io
import numpy as np
import pandas as pd

In [3]:
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [4]:
with io.open('data/corpus.txt','r',encoding='utf8') as f:
    corpus = f.readlines()

In [5]:
supreme = []
for tweet in corpus:
        supreme.append(tweet.split('|',2)[-2:])

In [6]:
def  change_value(strings):
    if strings == 'ironic':
        return int(1)
    else:
        return int(0)
        

In [7]:
corpus_array = np.array(supreme)

In [110]:
df_corpus = pd.DataFrame(data=corpus_array,columns=['cat','tweet'])

In [111]:
lel = df_corpus['cat'].apply(change_value)

In [112]:
df_corpus.insert(loc=2,column='category',value=lel)

In [113]:
tweets = df_corpus['tweet'].iloc[:]

In [114]:
df_corpus.head()

cat                                              tweet  category
0      ironic     No hay casi nadie de Cuenca en Benidorm 😆😆😆 \n         1
1      ironic  QuienLoHubieraDicho hacer tuneles es mas facil...         1
2  not_ironic  "La vida era madre fácil cuando no había Twitt...         0
3  not_ironic  "Oh no se para qué hablo con esta guachenga.. ...         0
4  not_ironic  "¿Qué fue lo primero que se te vino a la cabez...         0

In [115]:
tokenizer = Tokenizer(num_words=20000,lower=False)

In [116]:
tokenizer.fit_on_texts(tweets)
tokenizer.word_index
lista = tokenizer.texts_to_sequences(tweets)

In [117]:
df_corpus.insert(loc=3,column='tweet_sec', value=lista)

In [127]:
type(lista[0])

list

In [118]:
df_corpus.head()

cat                                              tweet  category  \
0      ironic     No hay casi nadie de Cuenca en Benidorm 😆😆😆 \n         1   
1      ironic  QuienLoHubieraDicho hacer tuneles es mas facil...         1   
2  not_ironic  "La vida era madre fácil cuando no había Twitt...         0   
3  not_ironic  "Oh no se para qué hablo con esta guachenga.. ...         0   
4  not_ironic  "¿Qué fue lo primero que se te vino a la cabez...         0   

                                           tweet_sec  
0         [33, 54, 259, 177, 2, 4493, 8, 9784, 9785]  
1  [9786, 82, 9787, 12, 45, 1666, 1, 82, 1667, 60...  
2           [59, 61, 151, 212, 719, 24, 6, 379, 561]  
3  [459, 6, 14, 25, 72, 526, 22, 19, 9790, 39, 52...  
4  [296, 100, 17, 271, 1, 14, 21, 354, 3, 4, 192,...

In [119]:
vector_corpus = df_corpus[['category','tweet_sec']]

In [121]:
vector_corpus.to_csv(path_or_buf='./data/corpus_vectorized.txt', sep='|',index=False)

In [129]:
vector_corpus.head()

category                                          tweet_sec
0         1         [33, 54, 259, 177, 2, 4493, 8, 9784, 9785]
1         1  [9786, 82, 9787, 12, 45, 1666, 1, 82, 1667, 60...
2         0           [59, 61, 151, 212, 719, 24, 6, 379, 561]
3         0  [459, 6, 14, 25, 72, 526, 22, 19, 9790, 39, 52...
4         0  [296, 100, 17, 271, 1, 14, 21, 354, 3, 4, 192,...

# LSTM

In [130]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

In [131]:
from sklearn.model_selection import train_test_split

In [132]:
max_features = 20000
maxlen = 20
batch_size = 32

In [134]:
tweet_train, tweet_test, label_train, label_test = \
train_test_split(vector_corpus['tweet_sec'], vector_corpus['category'], test_size=0.33)

In [150]:
print tweet_test.head()
print tweet_test.count()

4811                              [441, 364, 193, 13, 94]
8395    [62, 12, 4, 3964, 16, 215, 1, 54, 1038, 14, 21...
7883    [2521, 4332, 16, 734, 5, 8, 619, 3115, 6728, 1...
889       [451, 14, 9, 2737, 7, 3190, 32, 4755, 133, 771]
4520    [147, 9, 653, 15715, 29, 6, 9, 116, 79, 15716,...
Name: tweet_sec, dtype: object
5051


In [149]:
print tweet_train.head()
print tweet_train.count()

7676     [281, 815, 19, 96, 529, 1069, 5, 12, 862, 98, ...
13215    [6182, 126, 49, 4, 4718, 27, 665, 2, 94, 8, 11...
14494    [107, 77, 101, 9728, 48, 1245, 1270, 2982, 1, ...
5999          [7069, 1605, 17478, 17479, 24, 155, 12, 106]
15074    [60, 1865, 1130, 816, 182, 1447, 2282, 98, 67,...
Name: tweet_sec, dtype: object
10255


In [153]:
print('Pad sequences (samples x time)')
tweet_train = sequence.pad_sequences(tweet_train, maxlen=maxlen)
tweet_test = sequence.pad_sequences(tweet_test, maxlen=maxlen)
print('x_train shape:', tweet_train.shape)
print('x_test shape:', tweet_test.shape)

Pad sequences (samples x time)
('x_train shape:', (10255, 20))
('x_test shape:', (5051, 20))


In [154]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

Build model...


In [155]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [158]:
print('Train...')
model.fit(tweet_train, label_train,
          batch_size=batch_size,
          epochs=20,
          validation_data=(tweet_test, label_test))
score, acc = model.evaluate(tweet_test, label_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Train on 10255 samples, validate on 5051 samples
Epoch 1/20
10255/10255 [==============================] - 25s - loss: 0.2961 - acc: 0.8859 - val_loss: 0.2826 - val_acc: 0.8909
Epoch 2/20
10255/10255 [==============================] - 24s - loss: 0.1456 - acc: 0.9460 - val_loss: 0.3246 - val_acc: 0.8773
Epoch 3/20
10255/10255 [==============================] - 24s - loss: 0.0762 - acc: 0.9745 - val_loss: 0.3791 - val_acc: 0.8679
Epoch 4/20
10255/10255 [==============================] - 24s - loss: 0.0498 - acc: 0.9835 - val_loss: 0.4353 - val_acc: 0.8654
Epoch 5/20
10255/10255 [==============================] - 24s - loss: 0.0363 - acc: 0.9872 - val_loss: 0.5035 - val_acc: 0.8580
Epoch 6/20
10255/10255 [==============================] - 24s - loss: 0.0205 - acc: 0.9941 - val_loss: 0.5773 - val_acc: 0.8608
Epoch 7/20
10255/10255 [==============================] - 24s - loss: 0.0182 - acc: 0.9944 - val_loss: 0.6150 - val_acc: 0.8549
Epoch 8/20
10255/10255 [======================